# Data Science and STEM Salaries Analysis

---

## 1.0 Objectives

## 2.0 Dataset Information

### 2.1 Description

### 2.2 Data Dictionary

### 2.3 Acknowledgement & Source

---

<br/>

## 3.0 Import Library and Set Constant

In [1]:
# Enable graph in jupyter notebook
%matplotlib inline

# System
import sys
import os

# Fetch data
import requests

In [2]:
# EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Constant
LEVELS_FYI_SALARY_SOURCE_URL = 'https://www.levels.fyi/js/salaryData.json'
RAW_SALARY_FILE_URL = 'levels-fyi-salary-raw.csv'
DS_RAW_SALARY_FILE_URL = 'ds-levels-fyi-salary-raw.csv'
DS_CLEANED_SALARY_FILE_URL = 'ds-levels-fyi-salary-cleaned.csv'

UTILS_PATH = os.path.join('..', 'utils')

In [4]:
# Below functions are common function that the author wrote and used
# Statistics for Missing Value in DataFrame
sys.path.append(UTILS_PATH)

from stats_df_na import find_col_with_na, calculate_na_pct

<br />

## 4.0 Setting Graph Style

In [5]:
plt.rcParams["figure.figsize"] = (6, 3)
plt.rcParams['figure.dpi'] = 150

sns.set_style("white")

<br />

## 5.0 Fetch and Write Data

In [67]:
# Fetch data in JSON
data = requests.get('https://www.levels.fyi/js/salaryData.json').json()
json_df = pd.DataFrame(data)

# Output raw salary in CSV
json_df.to_csv(RAW_SALARY_FILE_URL, index=False)

<br />

In [6]:
df = pd.read_csv(RAW_SALARY_FILE_URL, parse_dates=['timestamp'], dayfirst=False)

In [7]:
df['title'].unique()

array(['Product Manager', 'Software Engineer',
       'Software Engineering Manager', 'Data Scientist',
       'Solution Architect', 'Technical Program Manager',
       'Human Resources', 'Product Designer', 'Marketing',
       'Business Analyst', 'Hardware Engineer', 'Sales', 'Recruiter',
       'Mechanical Engineer', 'Management Consultant'], dtype=object)

In [8]:
ds_df = df[ df['title'] == 'Data Scientist']

In [9]:
ds_df.to_csv(DS_RAW_SALARY_FILE_URL, index=False)

<br />

Below will import the csv as the main dataframe

In [10]:
df = pd.read_csv(DS_RAW_SALARY_FILE_URL)

<br />

## 6.0 Statistics Summary of Dataset

In [11]:
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber
0,2018-06-05 14:06:30,LinkedIn,Senior,Data Scientist,274,"San Francisco, CA",5,0,Data Analysis,149,118,7,Male,NaN,7419,807.0,813
1,2018-06-08 00:29:47,Amazon,L4,Data Scientist,147417,"Seattle, WA",1,1,NaN,79203,68214,0,Male,NaN,11527,819.0,845
2,2018-06-08 09:49:25,Microsoft,64,Data Scientist,291,"Seattle, WA",6,7,ML / AI,229,36,26,Male,NaN,11527,819.0,849
3,2018-06-08 17:55:09,ebay,26,Data Scientist,0,"San Jose, CA",6,5,NaN,0,0,0,Female,NaN,7422,807.0,855
4,2018-06-10 19:39:35,Twitter,Staff,Data Scientist,310,"San Francisco, CA",2,5,ML / AI,107,188,15,Male,NaN,7419,807.0,870


In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
totalyearlycompensation,2578.0,1386.326610,17763.562817,0.0,117.00,177.0,257.00,503119.0
yearsofexperience,2578.0,5.071373,4.607735,0.0,2.00,4.0,7.00,42.0
yearsatcompany,2578.0,2.064779,2.425945,0.0,1.00,1.0,3.00,22.0
basesalary,2578.0,904.607836,10754.643330,0.0,90.25,131.0,180.00,252028.0
stockgrantvalue,2578.0,390.232351,7462.262178,0.0,0.00,19.0,53.75,284257.0
bonus,2578.0,91.486424,1876.027778,0.0,1.00,13.0,24.00,70932.0
cityid,2578.0,9986.732739,6609.613358,1182.0,7419.00,8178.0,11521.00,47926.0
dmaid,2578.0,620.963150,275.901269,0.0,505.00,807.0,807.00,862.0
rowNumber,2578.0,43468.069822,24508.271158,813.0,22799.25,44403.0,65561.75,83867.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2578 entries, 0 to 2577
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                2578 non-null   object 
 1   company                  2578 non-null   object 
 2   level                    2572 non-null   object 
 3   title                    2578 non-null   object 
 4   totalyearlycompensation  2578 non-null   int64  
 5   location                 2578 non-null   object 
 6   yearsofexperience        2578 non-null   int64  
 7   yearsatcompany           2578 non-null   int64  
 8   tag                      2573 non-null   object 
 9   basesalary               2578 non-null   int64  
 10  stockgrantvalue          2578 non-null   int64  
 11  bonus                    2578 non-null   int64  
 12  gender                   1740 non-null   object 
 13  otherdetails             1815 non-null   object 
 14  cityid                  

<br/>

## 7.0 Preprocessing

### 7.1 Data Cleaning

Find which feature column contains missing values

In [14]:
col_with_na, has_na = find_col_with_na(df)

In [15]:
col_with_na

Index(['level', 'tag', 'gender', 'otherdetails'], dtype='object')

In [16]:
calculate_na_pct(df)

gender          32.505818
otherdetails    29.596587
level            0.232739
tag              0.193949
dtype: float64

<br />

In [18]:
df['gender'].value_counts()

Male      1357
Female     361
Other       22
Name: gender, dtype: int64

In [17]:
df['gender'].unique()

array(['Male', 'Female', nan, 'Other'], dtype=object)

The dataset consists of 32.51% missing value for feature *gender*. Since the user at levels.fyi does not intend to disclose, we will impute these missing value as 'other' category.

In [19]:
replace_value = {
    'gender': 'Other'
}

df = df.fillna(value=replace_value)

In [21]:
calculate_na_pct(df)

otherdetails    29.596587
level            0.232739
tag              0.193949
dtype: float64

<br />

For the feature *otherdetails* that consists 29.60% of missing value, we can ignore it for now because this feature will be drop during feature selection. Same goes for the feature *tag* that consists 0.19% missing value.

As for the last feature *level* that contains 0.23% of missing value, we can remove the instances from the dataset. Since, the missing value is small and can be safely ignored.

In [22]:
df = df.dropna(subset=['level'])

<br />

In [23]:
calculate_na_pct(df)

otherdetails    29.587869
tag              0.194401
dtype: float64

<br />

### 7.2 Feature Selection

In [24]:
df.columns

Index(['timestamp', 'company', 'level', 'title', 'totalyearlycompensation',
       'location', 'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'gender', 'otherdetails', 'cityid', 'dmaid',
       'rowNumber'],
      dtype='object')

We will be dropping the feature *title* as this is a filtered dataset contains only 'Data Scientist* as the only title. Moreover, we will also dropping feature *tag*, *otherdetails*, *cityid*, *dmaid* and *rowNumber*

In [27]:
df = df.drop(['title', 'tag', 'otherdetails', 'cityid', 'dmaid','rowNumber'], axis=1)

In [28]:
df.columns

Index(['timestamp', 'company', 'level', 'totalyearlycompensation', 'location',
       'yearsofexperience', 'yearsatcompany', 'basesalary', 'stockgrantvalue',
       'bonus', 'gender'],
      dtype='object')

<br />

In [29]:
calculate_na_pct(df)

Series([], dtype: float64)

After performing data cleaning and feature selection, the dataframe no longer contains any missing value and irrevelant features

<br />

## 8.0 EDA

## 9.0 Building Model

---

## 9.0 Conclusion